In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import itertools
import pickle

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")

columns_to_mantain = ['SESSO', 'ANNI', 'ALTEZZA', 'PESO', 'BMI', 'Mezzo', 'Tot Testa', 'Tot Torace', 'Tot Addome', 
                    'Tot Scheletro', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx', 'Omero dx', 'Radio dx', 'Ulna dx',
                    'Omero sx', 'Radio sx', 'Ulna sx', 'carpo dx', 'I metacarpo dx', 'II metacarpo dx', 'III metacarpo dx',
                    'IV  metacarpo dx', 'V metacarpo dx', 'I raggio dx', 'II raggio dx', 'III raggio dx',
                    'IV raggio dx', 'V raggio dx', 'carpo sx', 'I metacarpo sx', 'II metacarpo sx', 'III metacarpo sx',
                    'IV metacarpo sx', 'V metacarpo sx', 'I raggio sx', 'II raggio sx', 'III raggio sx', 'IV raggio sx',
                    'V raggio sx', 'Spalla dx', 'Spalla sx', 'Gomito dx', 'Gomito sx', 'Polso dx', 'Polso sx', 
                    'femore dx', 'tibia dx', 'fibula dx', 'rotula dx', 'femore sx', 'tibia sx', 'fibula sx', 
                    'rotula sx', 'tarso dx', 'I metatarso dx', 'II metatarso dx', 'III metatarso dx', 'IV metatarso dx',
                    'V metatarso dx', 'I raggio dx', 'II raggio dx', 'III raggio dx', 'IV raggio dx', 'V raggio dx', 
                    'tarso sx', 'I metatarso sx', 'II metatarso sx', 'III metatarso sx', 'IV metatarso sx', 
                    'V metatarso sx', 'I raggio sx', 'II raggio sx', 'III raggio sx', 'IV raggio sx', 'V raggio sx',
                    'Art. coxo-femorale dx', 'Art. coxo-femorale sx', 'Rotula o Ginocchio dx', 'Rotula o Ginocchio sx',
                    'Caviglia dx', 'Caviglia sx']

X = df.loc[:, columns_to_mantain]

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

print(X.shape)

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 115)
(130, 74)


In [3]:
def nested_cv_accuracy(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                    clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['n_estimators'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['max_features'] = params[2]

            clf = IsolationForest(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [4]:
results = nested_cv_accuracy(X)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.5,0.55,accuracy on nested cv,60.769231,10.146851,71.560144,16.31485,45.714286,11.605769,"tot columns, standard scaled"


In [5]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [6]:
def nested_cv_f1(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                    clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['n_estimators'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['max_features'] = params[2]

            clf = IsolationForest(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'f1 on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [7]:
scores_df = add_record(scores_df, nested_cv_f1(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.5,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.5,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"


In [8]:
def outer_cv_inner_holdout_accuracy(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}
    
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['n_estimators'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['max_features'] = params[2]

            clf = IsolationForest(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [9]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_accuracy(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.5,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.5,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"
2,IsolationForest,75,0.5,0.775,accuracy on outer cv inner holdout,63.076923,9.187935,72.332242,11.835588,50.000000,13.801311,"tot columns, standard scaled"


In [10]:
def outer_cv_inner_holdout_f1(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}
    
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['n_estimators'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['max_features'] = params[2]

            clf = IsolationForest(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [11]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_f1(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.500000,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.500000,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"
2,IsolationForest,75,0.500000,0.775,accuracy on outer cv inner holdout,63.076923,9.187935,72.332242,11.835588,50.000000,13.801311,"tot columns, standard scaled"
3,IsolationForest,75,0.064444,0.775,f1 on outer cv inner holdout,66.666667,8.268982,70.274964,9.147641,69.047619,14.907120,"tot columns, standard scaled"


In [12]:
def outer_holdout_inner_cv_accuracy(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0
        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['n_estimators'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['max_features'] = params[2]

        clf = IsolationForest(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [13]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_accuracy(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.500000,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.500000,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"
2,IsolationForest,75,0.500000,0.775,accuracy on outer cv inner holdout,63.076923,9.187935,72.332242,11.835588,50.000000,13.801311,"tot columns, standard scaled"
3,IsolationForest,75,0.064444,0.775,f1 on outer cv inner holdout,66.666667,8.268982,70.274964,9.147641,69.047619,14.907120,"tot columns, standard scaled"
4,IsolationForest,75,0.500000,0.100,accuracy on outer holdout inner cv,66.666667,7.903095,68.888889,16.405359,51.990493,12.389411,"tot columns, standard scaled"


In [14]:
def outer_holdout_inner_cv_f1(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0
        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_estimatorss, contaminations, max_featuress):
                clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['n_estimators'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['max_features'] = params[2]

        clf = IsolationForest(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [15]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_f1(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.500000,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.500000,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"
2,IsolationForest,75,0.500000,0.775,accuracy on outer cv inner holdout,63.076923,9.187935,72.332242,11.835588,50.000000,13.801311,"tot columns, standard scaled"
3,IsolationForest,75,0.064444,0.775,f1 on outer cv inner holdout,66.666667,8.268982,70.274964,9.147641,69.047619,14.907120,"tot columns, standard scaled"
4,IsolationForest,75,0.500000,0.100,accuracy on outer holdout inner cv,66.666667,7.903095,68.888889,16.405359,51.990493,12.389411,"tot columns, standard scaled"
5,IsolationForest,75,0.336667,1.000,f1 on outer holdout inner cv,60.256410,10.094882,58.974359,15.091219,52.350427,26.359243,"tot columns, standard scaled"


In [16]:
def double_holdout_accuracy(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0

        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(n_estimatorss, contaminations, max_featuress):
            clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['n_estimators'] = params[0]
                best_params['contamination'] = params[1]
                best_params['max_features'] = params[2]

        clf = IsolationForest(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [17]:
scores_df = add_record(scores_df, double_holdout_accuracy(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.500000,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.500000,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"
2,IsolationForest,75,0.500000,0.775,accuracy on outer cv inner holdout,63.076923,9.187935,72.332242,11.835588,50.000000,13.801311,"tot columns, standard scaled"
3,IsolationForest,75,0.064444,0.775,f1 on outer cv inner holdout,66.666667,8.268982,70.274964,9.147641,69.047619,14.907120,"tot columns, standard scaled"
4,IsolationForest,75,0.500000,0.100,accuracy on outer holdout inner cv,66.666667,7.903095,68.888889,16.405359,51.990493,12.389411,"tot columns, standard scaled"
5,IsolationForest,75,0.336667,1.000,f1 on outer holdout inner cv,60.256410,10.094882,58.974359,15.091219,52.350427,26.359243,"tot columns, standard scaled"
6,IsolationForest,75,0.500000,0.325,accuracy on double holdout,60.256410,7.903095,68.055556,5.196746,43.162393,7.125356,"tot columns, standard scaled"


In [18]:
def double_holdout_f1(X):
    n_estimatorss = np.arange(50, 201, 25)
    contaminations = np.linspace(0.01, 0.5, 10)
    max_featuress = np.linspace(0.1, 1.0, 5)

    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=3)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0

        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(n_estimatorss, contaminations, max_featuress):
            clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['n_estimators'] = params[0]
                best_params['contamination'] = params[1]
                best_params['max_features'] = params[2]

        clf = IsolationForest(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_params['n_estimators'],
            'best contamination': best_params['contamination'],
            'best max_features': best_params['max_features'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [19]:
scores_df = add_record(scores_df, double_holdout_f1(X))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,IsolationForest,50,0.500000,0.550,accuracy on nested cv,60.769231,10.146851,71.560144,16.314850,45.714286,11.605769,"tot columns, standard scaled"
1,IsolationForest,75,0.500000,0.325,f1 on nested cv,64.615385,8.092171,68.784922,10.226997,66.666667,17.431434,"tot columns, standard scaled"
2,IsolationForest,75,0.500000,0.775,accuracy on outer cv inner holdout,63.076923,9.187935,72.332242,11.835588,50.000000,13.801311,"tot columns, standard scaled"
3,IsolationForest,75,0.064444,0.775,f1 on outer cv inner holdout,66.666667,8.268982,70.274964,9.147641,69.047619,14.907120,"tot columns, standard scaled"
4,IsolationForest,75,0.500000,0.100,accuracy on outer holdout inner cv,66.666667,7.903095,68.888889,16.405359,51.990493,12.389411,"tot columns, standard scaled"
5,IsolationForest,75,0.336667,1.000,f1 on outer holdout inner cv,60.256410,10.094882,58.974359,15.091219,52.350427,26.359243,"tot columns, standard scaled"
6,IsolationForest,75,0.500000,0.325,accuracy on double holdout,60.256410,7.903095,68.055556,5.196746,43.162393,7.125356,"tot columns, standard scaled"
7,IsolationForest,50,0.173333,0.325,f1 on double holdout,67.948718,1.813094,64.886364,19.560204,72.941176,19.139505,"tot columns, standard scaled"


In [20]:
file_path = "if_exp4_df.pickle"

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)